# 11_1 In class example, Classification with questions
We will load the breast cancer dataset and figure out what is the most important predictor(s) for breast cancer. And then see what happens to the prognosis when these predictors are permuted.

<mark>Note:  Any data gleaned from this sort of thing is only as good as your model.  So if your model is very bad at making predictions then it's notions about what are the most important predictors are also very bad!

<!--See <a href="https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance_multicollinear.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-multicollinear-py">this</a> page for a different take on removing correlated columns-->

In [29]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

# Load Data

In [30]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
# X, y = data.data, data.target
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [31]:
#want to see it all in a dataframe?
df = pd.DataFrame(data= np.c_[data['data']],
                     columns= data['feature_names'])
df.head()
y=pd.Series(data['target'])